### Importing Libraries

In [24]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import nltk
import time

In [35]:
i = 2
list_reviews = []
URL = "https://www.amazon.it/Astroworld-2-LP-Travis-Scott/product-reviews/B07FZ99HXL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"

while i < 4:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
    }
    webpage = requests.get(URL, headers=headers)
    soup = BeautifulSoup(webpage.content, 'html.parser')
    list_reviews.append(soup.find_all("span", {'data-hook': 'review-body'}))

    URL = f"https://www.amazon.it/Astroworld-2-LP-Travis-Scott/product-reviews/B07FZ99HXL/ref=cm_cr_getr_d_paging_btm_next_{str(i)}?ie=UTF8&reviewerType=all_reviews&pageNumber={str(i)}"

    i += 1
    time.sleep(30)


In [36]:
list_reviews

[[<span class="a-size-base review-text review-text-content" data-hook="review-body">
  <span>Ottimo disco, per chi ama il genere hip hop, da avere.arrivato nei tempi prestabiliti.</span>
  </span>,
  <span class="a-size-base review-text review-text-content" data-hook="review-body">
  <span>Bel cd lo consiglio</span>
  </span>,
  <span class="a-size-base review-text review-text-content" data-hook="review-body">
  <span>Preso come regalo per un collezionista di vinili. Confezione integra, buon prezzo, migliore di altri trovati in rete.</span>
  </span>,
  <span class="a-size-base review-text review-text-content" data-hook="review-body">
  <span>Quello ke voleva Mio figlio</span>
  </span>,
  <span class="a-size-base review-text review-text-content" data-hook="review-body">
  <span>tenuto bene, buona consegna anche in anticipo</span>
  </span>,
  <span class="a-size-base review-text review-text-content" data-hook="review-body">
  <span>Servizio Amazon efficiente come sempre ma il vinile è

In [21]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
}
URL = "https://www.amazon.it/Astroworld-2-LP-Travis-Scott/product-reviews/B07FZ99HXL/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3"
webpage = requests.get(URL, headers=headers)
soup = BeautifulSoup(webpage.content, 'html.parser')
rev = soup.find_all("span", {'data-hook': 'review-body'})

In [22]:
rev

[]